In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
data_new=pd.read_csv('C:\\Users\\admin-l02\\Desktop\\dataset\\data_new.csv',encoding='gbk')
data_new.drop('source',axis=1,inplace=True)#source列值都是xs 可删除该特征

y_features=[]#存放y特征
x_features=[]#存放x特征
for col in data_new.columns:
    if col=='status':
        y_features.append(col)
    else:
        x_features.append(col)

X=data_new.loc[:,x_features]
y=data_new.loc[:,y_features]


#使用随机森林选取特征
#平均不纯度减少 mean decrease impurity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)#划分数据集
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
feature1 = sorted(zip(map(lambda x: '%.4f'%x, rf.feature_importances_), list(X_train.columns)), reverse=True)

# 平均精确率减少 Mean decrease accuracy
import numpy as np
from collections import defaultdict
from sklearn.model_selection import cross_val_score, ShuffleSplit

from sklearn.metrics import r2_score

scores = defaultdict(list)
rs = ShuffleSplit(n_splits=5, test_size=0.3, random_state=2018)
for train_idx, test_idx in rs.split(X_train):
    x_train, x_test = X_train.values[train_idx], X_train.values[test_idx]
    Y_train, Y_test = y_train.values[train_idx], y_train.values[test_idx]
    r = rf.fit(x_train, Y_train)
    acc = r2_score(Y_test, rf.predict(x_test))
    for i in range(x_train.shape[1]):
        X_t = x_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[X_train.columns[i]].append((acc - shuff_acc) / acc)

feature2 = sorted([('%.4f' % abs(np.mean(score)), feat) for feat, score in scores.items()], reverse=True)

#根据之前两个算的相关性的值，综合选择特征，选择两种排列后四十个的数据 作比较
drop_feature = []#要删除的特征
for feature in X_train.columns:
    if feature in [t[1] for t in feature1[50:]] and feature in [t[1] for t in feature2[50:]]:
        drop_feature.append(feature)

X_train.drop(drop_feature, axis = 1, inplace = True)
X_test.drop(drop_feature, axis = 1, inplace = True)

In [ ]:
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)

from sklearn.metrics import accuracy_score, roc_auc_score


def model_metrics(clf, X_train, X_test, y_train, y_test):
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    y_train_proba = clf.predict_proba(X_train)[:, 1]
    y_test_proba = clf.predict_proba(X_test)[:, 1]

    # 准确率
    print('[准确率]', end=' ')
    print('训练集：', '%.4f' % accuracy_score(y_train, y_train_pred), end=' ')
    print('测试集：', '%.4f' % accuracy_score(y_test, y_test_pred))

    # auc取值：用roc_auc_score或auc
    print('[auc值]', end=' ')
    print('训练集：', '%.4f' % roc_auc_score(y_train, y_train_proba), end=' ')
    print('测试集：', '%.4f' % roc_auc_score(y_test, y_test_proba))

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
#from lightgbm.sklearn import LGBMClassifier


lr = LogisticRegression()#LR模型
param = {'C': [1e-3,0.01,0.1,1,10,100,1e3], 'penalty':['l1', 'l2']}

gsearch = GridSearchCV(lr, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

lr = LogisticRegression(C = 0.1, penalty = 'l1')
lr.fit(X_train, y_train)
model_metrics(lr, X_train, X_test, y_train, y_test)

最佳参数： {'C': 0.1, 'penalty': 'l1'}
训练集的最佳分数： 0.790700810106611
测试集的最佳分数： 0.7803694198407979
[准确率] 训练集： 0.8016 测试集： 0.7870
[auc值] 训练集： 0.8101 测试集： 0.7804


Svm 模型

In [11]:
# 线性SVM
svm_linear = svm.SVC(kernel = 'linear', probability=True)
param = {'C':[0.01,0.1,1]}
gsearch = GridSearchCV(svm_linear, param_grid = param,scoring ='roc_auc', cv=5)
gsearch.fit(X_train, y_train)

print('最佳参数：',gsearch.best_params_)
print('训练集的最佳分数：', gsearch.best_score_)
print('测试集的最佳分数：', gsearch.score(X_test, y_test))

最佳参数： {'C': 0.01}
训练集的最佳分数： 0.794634380365573
测试集的最佳分数： 0.7816630674052977


In [12]:
svm_linear = svm.SVC(C = 0.01, kernel = 'linear', probability=True)
svm_linear.fit(X_train, y_train)
model_metrics(svm_linear, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7986 测试集： 0.7786
[auc值] 训练集： 0.8121 测试集： 0.7817


In [13]:
param = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=8,min_samples_split=300,min_samples_leaf=20, max_features='sqrt',random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_depth': 11, 'min_samples_split': 100}, 0.7096312461899095)

In [14]:
param = {'min_samples_split':range(50,1000,100), 'min_samples_leaf':range(60,101,10)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=100,min_samples_leaf=20, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 100, 'min_samples_split': 50}, 0.7050112914709954)

In [15]:
param = {'max_features':range(7,20,2)}
gsearch = GridSearchCV(DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80, max_features='sqrt', random_state =2333),
                       param_grid = param,scoring ='roc_auc', cv=5)

gsearch.fit(X_train, y_train)
# gsearch.grid_scores_, 
gsearch.best_params_, gsearch.best_score_

({'max_features': 11}, 0.7196633559313361)

In [16]:
dt = DecisionTreeClassifier(max_depth=11,min_samples_split=550,min_samples_leaf=80,max_features=19, random_state =2333)
dt.fit(X_train, y_train)
model_metrics(dt, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.7493 测试集： 0.7484
[auc值] 训练集： 0.7267 测试集： 0.6964


In [17]:

xgb0 = XGBClassifier()
xgb0.fit(X_train, y_train)

model_metrics(xgb0, X_train, X_test, y_train, y_test)

[准确率] 训练集： 0.8521 测试集： 0.7835
[auc值] 训练集： 0.9158 测试集： 0.7709


C:\Users\admin-l02\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\admin-l02\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
